In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer-satisfaction-in-airline/Invistico_Airline.csv


**Part III**

Now, it's time for a random forest. 

**Machine Learning Models Scores**

1. Binomial Logistic Regression- 80.2% Accuracy

2. Tuned Decision Tree- 94.1% Accuracy

In [2]:
# Load in the dataset
invistico= pd.read_csv('/kaggle/input/customer-satisfaction-in-airline/Invistico_Airline.csv')

In [3]:
# Import necessary libraries and packages
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
invistico.head()

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,...,4,2,2,0,2,4,2,5,0,0.0


In [5]:
# View datatypes
invistico.dtypes

satisfaction                          object
Customer Type                         object
Age                                    int64
Type of Travel                        object
Class                                 object
Flight Distance                        int64
Seat comfort                           int64
Departure/Arrival time convenient      int64
Food and drink                         int64
Gate location                          int64
Inflight wifi service                  int64
Inflight entertainment                 int64
Online support                         int64
Ease of Online booking                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Cleanliness                            int64
Online boarding                        int64
Departure Delay in Minutes             int64
Arrival Delay in Minutes             float64
dtype: obj

In [6]:
# Check for missing values
invistico.isna().sum()

satisfaction                           0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Seat comfort                           0
Departure/Arrival time convenient      0
Food and drink                         0
Gate location                          0
Inflight wifi service                  0
Inflight entertainment                 0
Online support                         0
Ease of Online booking                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Cleanliness                            0
Online boarding                        0
Departure Delay in Minutes             0
Arrival Delay in Minutes             393
dtype: int64

In [7]:
# Drop missing values
invistico= invistico.dropna(axis=0)

In [8]:
# Confirm missing valus are dropped
invistico.isna().sum()

satisfaction                         0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Seat comfort                         0
Departure/Arrival time convenient    0
Food and drink                       0
Gate location                        0
Inflight wifi service                0
Inflight entertainment               0
Online support                       0
Ease of Online booking               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Cleanliness                          0
Online boarding                      0
Departure Delay in Minutes           0
Arrival Delay in Minutes             0
dtype: int64

In [9]:
# Use OneHotEncoding to convert categorical features
invistico= pd.get_dummies(invistico, columns= ['Customer Type', 'Type of Travel', 'Class'])

In [10]:
# View data
invistico.head()

,satisfaction,Age,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,...,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus
0,satisfied,65,265,0,0,0,2,2,4,2,...,2,0,0.0,True,False,False,True,False,True,False
1,satisfied,47,2464,0,0,0,3,0,2,2,...,2,310,305.0,True,False,False,True,True,False,False
2,satisfied,15,2138,0,0,0,3,2,0,2,...,2,0,0.0,True,False,False,True,False,True,False
3,satisfied,60,623,0,0,0,3,3,4,3,...,3,0,0.0,True,False,False,True,False,True,False
4,satisfied,70,354,0,0,0,3,4,3,4,...,5,0,0.0,True,False,False,True,False,True,False


In [11]:
# Check datatypes
invistico.dtypes

satisfaction                          object
Age                                    int64
Flight Distance                        int64
Seat comfort                           int64
Departure/Arrival time convenient      int64
Food and drink                         int64
Gate location                          int64
Inflight wifi service                  int64
Inflight entertainment                 int64
Online support                         int64
Ease of Online booking                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Cleanliness                            int64
Online boarding                        int64
Departure Delay in Minutes             int64
Arrival Delay in Minutes             float64
Customer Type_Loyal Customer            bool
Customer Type_disloyal Customer         bool
Type of Travel_Business travel          bool
Type of Tr

In [12]:
# Define variables for model
y= invistico['satisfaction']
X= invistico.copy()
X= X.drop('satisfaction', axis=1)

In [13]:
# Train,test and validate set
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.25, random_state= 0)

X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train, test_size= 0.25, random_state= 0)

In [14]:
# Hyperparameters
cv_params= {'n_estimators': [50,100],
            'max_depth': [10,50],
            'min_samples_leaf': [0.5,1],
            'min_samples_split': [0.001,0.01],
            'max_features': ['sqrt'],
            'max_samples': [0.5,0.9]
           }

In [15]:
# Create list of split indices
split_index= [0 if x in X_val.index else -1 for x in X_train.index]
custom_split= PredefinedSplit(split_index)

In [16]:
# Instantiate the model
rf= RandomForestClassifier(random_state= 0)

In [17]:
# Search over specified parameters
rf_val= GridSearchCV(rf, cv_params, cv= custom_split, refit= 'f1', n_jobs= -1, verbose= 1)

In [18]:
%%time
# Fit the model
rf_val.fit(X_train, y_train)

Fitting 1 folds for each of 32 candidates, totalling 32 fits
CPU times: user 9.27 s, sys: 235 ms, total: 9.51 s
Wall time: 59.5 s


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ..., -1, -1])),
             estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [10, 50], 'max_features': ['sqrt'],
                         'max_samples': [0.5, 0.9],
                         'min_samples_leaf': [0.5, 1],
                         'min_samples_split': [0.001, 0.01],
                         'n_estimators': [50, 100]},
             refit='f1', verbose=1)

In [19]:
# Best parameters
rf_val.best_params_

{'max_depth': 50,
 'max_features': 'sqrt',
 'max_samples': 0.9,
 'min_samples_leaf': 1,
 'min_samples_split': 0.001,
 'n_estimators': 50}

In [20]:
# Use optimal parameters on GridsearchCV
rf_opt= RandomForestClassifier(n_estimators= 50, max_depth= 50, min_samples_leaf= 1, min_samples_split= 0.001, max_features= 'sqrt', max_samples= 0.9, random_state= 0)

In [21]:
%%time
# Fit the optimal model
rf_opt.fit(X_train, y_train)

CPU times: user 8.8 s, sys: 5.09 ms, total: 8.81 s
Wall time: 8.81 s


RandomForestClassifier(max_depth=50, max_samples=0.9, min_samples_split=0.001,
                       n_estimators=50, random_state=0)

In [22]:
# Predict on test set
y_pred= rf_opt.predict(X_test)

In [23]:
# Precision score
pc_test= precision_score(y_test, y_pred, pos_label= 'satisfied')
print('Precision score {pc:.3f}'.format(pc= pc_test))

Precision score 0.950


In [24]:
# Accuracy score
ac_test= accuracy_score(y_test, y_pred)
print('Accuracy score {ac:.3f}'.format(ac= ac_test))

Accuracy score 0.942


In [25]:
# Recall score
rc_test= recall_score(y_test, y_pred, pos_label= 'satisfied')
print('Recall score {rc:.3f}'.format(rc= rc_test))

Recall score 0.945


In [26]:
# F1 score
f1_test= f1_score(y_test, y_pred, pos_label= 'satisfied')
print('F1 score {f1:.3f}'.format(f1= f1_test))

F1 score 0.947


In [27]:
# Precision score on test data
print('\nThe precision score is {pc:.3f}'.format(pc= pc_test), 'for the test set,', '\nwhich means of all the positive predictions,', '{pc_pct:.1f}% of the predictions are true positive'.format(pc_pct= pc_test * 100))


The precision score is 0.950 for the test set, 
which means of all the positive predictions, 95.0% of the predictions are true positive


In [28]:
# Accuracy score on test data
print('\nThe accuracy score is {ac:.3f}'.format(ac= ac_test), 'for the test set,', '\nwhich means of all the cases in the test set,', '{ac_pct:.1f}% are predicted true positive or true negative'.format(ac_pct= ac_test * 100))


The accuracy score is 0.942 for the test set, 
which means of all the cases in the test set, 94.2% are predicted true positive or true negative


In [29]:
# Recall score on the test set
print('\nThe recall score is {rc:.3f}'.format(rc= rc_test), 'for the test set,', '\nwhich means of all the real positive cases in the test set', '{rc_pct:.1f}% are predicted positive'.format(rc_pct= rc_test * 100))


The recall score is 0.945 for the test set, 
which means of all the real positive cases in the test set 94.5% are predicted positive


In [30]:
# F1 score on the test set
print('\nThe F1 score is {f1:.3f}'.format(f1= f1_test), 'for the test set,', '\nwhich means that the test set has a harmonic mean of {f1_pct:.1f}%'.format(f1_pct= f1_test * 100))


The F1 score is 0.947 for the test set, 
which means that the test set has a harmonic mean of 94.7%


**Scores**

The model performs well according to all 4 performance metrics. The model’s precision score is slightly better than the 3 other metrics.

In [31]:
# Create a table of results
table= pd.DataFrame({'Model': ['Binomial Logistic Regression', 'Tuned Decision Tree', 'Tuned Random Forest'],
                     'Accuracy': [0.801529, 0.940864, ac_test],
                     'Precision': [0.816142, 0.955197, pc_test],
                     'Recall': [0.821530, 0.935863, rc_test],
                     'F1 Score': [0.818827, 0.945422, f1_test]
                    })

table

,Model,Accuracy,Precision,Recall,F1 Score
0,Binomial Logistic Regression,0.801529,0.816142,0.821530,0.818827
1,Tuned Decision Tree,0.940864,0.955197,0.935863,0.945422
2,Tuned Random Forest,0.942450,0.950128,0.944501,0.947306


**Model Comparisons (so far)**

The Binomial Logistic Regression model achieved an accuracy of 0.801529, precision of 0.816142, recall of 0.821530, and an F1 score of 0.818827. While decent, these metrics indicate room for improvement compared to the other models evaluated.


The Tuned Decision Tree model performed significantly better with an accuracy of 0.940864, precision of 0.955197, recall of 0.935863, and an F1 score of 0.945422. This suggests the decision tree was able to capture the patterns in the data quite effectively after tuning.

However, the Tuned Random Forest model slightly edged it out as the top performer with an accuracy of 0.942450, precision of 0.950128, recall of 0.944501, and an F1 score of 0.947306. The random forest's ensemble learning approach allowed it to achieve the highest metrics overall, while still maintaining a relatively parsimonious model compared to an extremely complex single decision tree. Based on these results, the Tuned Random Forest is recommended as the strongest candidate model for deployment (for now). 

**Brief Summary for Stakeholders**

The random forest model predicted satisfaction with more than 94.2% accuracy. The precision is over 95% and the recall is approximately 94.5%.

The random forest model outperformed the tuned decision tree with the best hyperparameters inmost of the four scores. This indicates that the random forest model may perform better.